In [24]:
import json
from pathlib import Path

import folium
import matplotlib
import numpy as np
import pandas as pd
from exif import Image

In [34]:
BASE_LOC = Path("/media/pav/Storage/Photo/Phone")
file = Path("IMAG0036.jpg")

In [ ]:
def read_exif_data(file_path: Path) -> Image:
    """Read metadata from photo."""
    with open(file_path, 'rb') as f:
        return Image(f)        

In [26]:
img = read_exif_data(base_loc / file)
print('\n'.join([i for i in img.list_all() 
                 if i.startswith('gps_')]))

gps_latitude_ref
gps_latitude
gps_longitude_ref
gps_longitude
gps_altitude_ref
gps_altitude
gps_timestamp
gps_processing_method
gps_datestamp


In [27]:
img['gps_altitude_ref']

<GpsAltitudeRef.ABOVE_SEA_LEVEL: 0>

In [28]:
img['gps_processing_method']

/home/pav/python/venv/photo_location/lib/python3.10/site-packages/exif/_image.py:127: RuntimeWarning: ASCII tag contains 0 fewer bytes than specified
  return self.__getattr__(item)


'ASCII\x00\x00\x00NETWORK'

In [5]:
def get_images_from_folder(folder: Path) -> list[Path]:
    return [f for f in folder.rglob('*.jpg')]

In [30]:
def convert_coords_to_decimal(coords: tuple[float,...], ref: str) -> float:
    """Covert a tuple of coordinates in the format (degrees, minutes, seconds)
    and a reference to a decimal representation.

    Args:
        coords (tuple[float,...]): A tuple of degrees, minutes and seconds
        ref (str): Hemisphere reference of "N", "S", "E" or "W".

    Returns:
        float: A signed float of decimal representation of the coordinate.
    """
    if ref.upper() in ['W', 'S']:
        mul = -1
    elif ref.upper() in ['E', 'N']:
        mul = 1
    else:
        print("Incorrect hemisphere reference. "
              "Expecting one of 'N', 'S', 'E' or 'W', "
              f'got {ref} instead.')
        
    return mul * (coords[0] + coords[1] / 60 + coords[2] / 3600)   

In [31]:
def get_decimal_coord_from_exif(exif_data: Image
                                ) -> tuple[float, ...]:
    """Get coordinate data from exif and convert to a tuple of 
    decimal latitude, longitude and altitude.

    Args:
        exif_data (Image): exif Image object

    Returns:
        tuple[float, ...]: A tuple of decimal coordinates (lat, lon, alt)
    """
    try:
        lat = convert_coords_to_decimal(
            exif_data['gps_latitude'], 
            exif_data['gps_latitude_ref']
            )
        lon = convert_coords_to_decimal(
            exif_data['gps_longitude'], 
            exif_data['gps_longitude_ref']
            )
        alt = exif_data['gps_altitude']
        return (lat, lon, alt)
    except (AttributeError, KeyError):
        print('Image does not contain spatial data or data is invalid.')  
        raise

In [42]:
def read_spatial_data_from_folder(
    folder: Path, 
    image_extension: str = '*.jpg'
    ) -> dict[str, dict]:
    """Create a dictionary of spatial data from photos in a folder.

    Args:
        folder (Path): folder as a Path object
        image_extension (str): extension of images to read. 
            Defaults to '.jpg'.

    Returns:
        dict[str, dict]: A dictionary with filename as the key
            and a value of a dictionary if the format:
                {
                    'coordinates': tuple[float, ...],
                    'timestamp': str
                }
    """
    coord_dict = dict()
    source_files = [f for f in folder.rglob(image_extension)]
    exif = [read_exif_data(f) for f in source_files]
    
    for f, data in zip(source_files, exif):
        try:
            coord = get_decimal_coord_from_exif(data)
        except (AttributeError, KeyError):
            continue
        else:
            coord_dict[str(f)] = dict()
            coord_dict[str(f)]['latitude'] = coord[0]
            coord_dict[str(f)]['longitude'] = coord[1]
            coord_dict[str(f)]['altitude'] = coord[2]
        # Also read date when photo was taken (if available)
        try:
            coord_dict[str(f)]['timestamp'] = data.datetime
        except (AttributeError, KeyError):
            print(f"Photo {f.name} does not contain datetime information.")
            coord_dict[str(f)]['timestamp'] = None
    
    return coord_dict

In [43]:
res = read_spatial_data_from_folder(BASE_LOC)

/home/pav/python/venv/photo_location/lib/python3.10/site-packages/exif/_app1_metadata.py:425: RuntimeWarning: skipping bad IFD 0
  warnings.warn(f"skipping bad IFD {ifd_key}", RuntimeWarning)


Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is invalid.
Image does not contain spatial data or data is i

In [44]:
print(json.dumps(res, indent=4))

{
    "/media/pav/Storage/Photo/Phone/IMAG0036.jpg": {
        "latitude": -37.79912566666666,
        "longitude": 144.9850463611111,
        "altitude": 0.0,
        "timestamp": "2014:04:12 18:14:59"
    },
    "/media/pav/Storage/Photo/Phone/IMAG0037.jpg": {
        "latitude": -37.79912566666666,
        "longitude": 144.9850463611111,
        "altitude": 0.0,
        "timestamp": "2014:04:12 18:15:36"
    },
    "/media/pav/Storage/Photo/Phone/IMAG0038.jpg": {
        "latitude": -37.799106583333334,
        "longitude": 144.9850158611111,
        "altitude": 0.0,
        "timestamp": "2014:04:12 18:16:30"
    },
    "/media/pav/Storage/Photo/Phone/IMAG0039.jpg": {
        "latitude": -37.799106583333334,
        "longitude": 144.9850158611111,
        "altitude": 0.0,
        "timestamp": "2014:04:12 18:16:52"
    },
    "/media/pav/Storage/Photo/Phone/IMAG0042.jpg": {
        "latitude": -37.7990875,
        "longitude": 144.9850158611111,
        "altitude": 0.0,
        "time

In [48]:
df = pd.DataFrame(res).T
df['timestamp'] = pd.to_datetime(df.timestamp, format="%Y:%m:%d %H:%M:%S")
df.sort_values('timestamp', inplace=True)
df.head(5)

,latitude,longitude,altitude,timestamp
/media/pav/Storage/Photo/Phone/IMAG0036.jpg,-37.799126,144.985046,0.0,2014-04-12 18:14:59
/media/pav/Storage/Photo/Phone/IMAG0037.jpg,-37.799126,144.985046,0.0,2014-04-12 18:15:36
/media/pav/Storage/Photo/Phone/IMAG0038.jpg,-37.799107,144.985016,0.0,2014-04-12 18:16:30
/media/pav/Storage/Photo/Phone/IMAG0039.jpg,-37.799107,144.985016,0.0,2014-04-12 18:16:52
/media/pav/Storage/Photo/Phone/IMAG0042.jpg,-37.799087,144.985016,0.0,2014-04-12 18:41:07


In [49]:
# get point colors
cmap = matplotlib.cm.get_cmap('plasma')
norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
df['color_mapping'] = np.linspace(0, 1, num=df.shape[0])

In [47]:
def rgba_to_hex(rgba: tuple[float, ...]):
    return ('#{:02X}{:02X}{:02X}').format(*rgba[:3])


In [50]:
folium.Map()

In [52]:
# calculate bounds SW corner and NE corner, add 3 degrees so that everything fits
sw = (df.latitude.max() + 3, df.longitude.min() - 3)
ne = (df.latitude.min() - 3, df.longitude.max() + 3)

In [51]:
m = folium.Map(location=[df.latitude.mean(), df.longitude.mean()])



# Add markers
for lat, lon, col, date in zip(df.latitude.values, df.longitude.values, df.color_mapping.values, df.timestamp.values):
    folium.CircleMarker(
        [lat, lon], 
        color=rgba_to_hex(cmap(col, bytes=True)), 
        fill_color=rgba_to_hex(cmap(col, bytes=True)),
        radius=6,
        tooltip = np.datetime_as_string(date, unit='m')
        ).add_to(m)

m.fit_bounds([sw, ne])
m